In [3]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.metrics import *
from glob import glob
from sklearn.model_selection import train_test_split
from ipynb.fs.full.model import *
from ipynb.fs.full.utils import *
from ipynb.fs.full.metrics import *

In [4]:
def read_image(x):
    x = x.decode()
    image = cv2.imread(x, cv2.IMREAD_COLOR)
    image = np.clip(image - np.median(image)+127, 0, 255)
    image = image/255.0
    image = image.astype(np.float32)
    return image

In [5]:
def read_mask(y):
    y = y.decode()
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    mask = mask/255.0
    mask = mask.astype(np.float32)
    mask = np.expand_dims(mask, axis=-1)
    return mask

In [6]:
def parse_data(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        y = np.concatenate([y, y], axis=-1)
        return x, y
    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([384, 512, 3])
    y.set_shape([384, 512, 2])
    return x, y

In [7]:
def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=32)
    dataset = dataset.map(map_func=parse_data)
    dataset = dataset.repeat()
    dataset = dataset.batch(batch)
    return dataset

In [9]:
np.random.seed(42)
tf.random.set_seed(42)
create_dir("files")

path = "E:\\Teknofest\\Saglik\\Model1\\"

train_x = sorted(glob(os.path.join(path, "image_split_trainx", "*.png")))
train_y = sorted(glob(os.path.join(path, "mask_split_trainy", "*.png")))

train_x, train_y = shuffling(train_x, train_y)

valid_x = sorted(glob(os.path.join(path, "image_split_validx", "*.png")))
valid_y = sorted(glob(os.path.join(path, "mask_split_validy", "*.png")))

model_path = "files/model.h5"
batch_size = 16
epochs = 300
lr = 1e-4
shape = (384, 512, 3)

model = build_model(shape)
metrics = [
        dice_coef,
        iou,
        Recall(),
        Precision()
    ]
    
train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)
    
model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=metrics)

callbacks = [
        ModelCheckpoint(model_path),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=20),
        CSVLogger("files/data.csv"),
        TensorBoard(),
        EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=False)
    ]

train_steps = (len(train_x)//batch_size)
valid_steps = (len(valid_x)//batch_size)

if len(train_x) % batch_size != 0:
    train_steps += 1

if len(valid_x) % batch_size != 0:
    valid_steps += 1

model.fit(train_dataset,
            epochs=epochs,
            validation_data=valid_dataset,
            steps_per_epoch=train_steps,
            validation_steps=valid_steps,
            callbacks=callbacks,
            shuffle=False)

80142336/80134624 [==============================] - 59s 1us/step
Epoch 1/300
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unexpected EOF while parsing (<unknown>, line 1)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unexpected EOF while parsing (<unknown>, line 1)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unexpected EOF while parsing (<unknown>, line 1)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_

InvalidArgumentError:  Incompatible shapes at component 0: expected [?,384,512,3] but got [16,512,512,3].
	 [[node IteratorGetNext (defined at <ipython-input-9-1f9fca53c019>:51) ]] [Op:__inference_train_function_14771]

Function call stack:
train_function
